# Regresión Básica: Predecir eficiencia de gasolina

En este notebook, utilizaremos un dataset de coches [Auto MPG](https://archive.ics.uci.edu/ml/datasets/auto+mpg) para construir un modelo de redes neuronales con el que  predecir el consumo de vehículos de 1970 y 1980. En este dataset dispondremos de atributos como Cilindros, desplazamiento, potencia y peso. El objetivo de este ejercicio es predecir las millas por galón de combustible (MPG).

El set de datos esta disponible de el siguiente repositorio [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/).


Empezaremos importando las librerías principales:

In [1]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.4.1


### Leyendo los datos

En primer lugar, lo que deberemos hacer es descargar el dataset:

In [2]:
dataset_path = keras.utils.get_file("auto-mpg.data",
                                    "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path

'C:\\Users\\TheBridge\\.keras\\datasets\\auto-mpg.data'

Y lo leemos con pandas:

In [3]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


### Limpiando los datos

Trata los missings y variables categóricas:

### Divide los datos en train y test

Ahora divide el dataset en train y test, donde este último sea del 80%.

Llama a los datasets ``train_dataset`` y ``test_dataset``:

### Inspecciona los datos

Revisa rápidamente la distribucion conjunta del dataset de entrenamiento mediante un grid de gráficos. Repasa los gráficos de seaborn para ver cuál nos puede ofrecer esta visión, donde la diagonal principal muestre el *kernel density estimate*:

Muestra también los estadísticos principales del dataset:

### Separa las features del target

En este momento, tenemos juntos tanto las variables independientes como el target. Sepáralos para tener ``train_dataset`` y ``test_dataset`` con las variables independientes, y ``train_labels`` y ``test_labels`` como target:

### Estandariza los datos

Es una buena práctica estandarizar los datasets con variables de diferentes escalas y rangos. Aunque el modelo podría converger sin estandarizar, dificulta el entrenamiento y hace que el modelo resultante dependa de la elección de las unidades utilizadas en la entrada. Estandariza los datos sobreescribiendo los datos ``train_dataset`` y ``test_dataset``:

# Modelo

### Construye el modelo

Construyamos nuestro modelo. Aquí, utilizaremos un modelo `secuencial` con dos capas ocultas densamente conectadas y una capa de salida que devuelva un único valor continuo.

Por tanto, necesitamos construir un modelo con tres capas:
  * **Entrada**: con activación relu.
  * **Hidden layer**: con activación relu
  * **Salida**: será de regresión, por lo que se compondrá de una única neurona.
  
Pon las neuronas que consideres para las dos primeras capas, por ejemplo, 64. Después iteraremos con diferentes combinaciones.

Para el compile utiliza un ``loss='mse'``, un ``optimizer = `tf.keras.optimizers.RMSprop(0.001)` ``, y en ``metrics`` utiliza una lista con el `mae` y `mse`.

### Inspecciona el modelo

Utiliza uno de los métodos que hemos visto para obtener una descripción simple del modelo:

### Entrenar el modelo

Entrena el modelo para 1000 epochs y guarda los resultados del entrenamiento en una variable llamada `history`.
Emplea en el entrenamiento un 20% de los datos para validación, mediante el argumento `validation_split`.

Visualiza el progreso de entrenamiento del modelo usando las estadísticas almacenadas en el objeto `history.history`, donde representes el error de train con el de validación:

¿Alguna conclusión respecto a este gráfico?

Añade un early stopping al modelo. Ya hemos visto cómo hacerlo mediante el parámetro ``callback``. En este caso, pon un ``patience`` de 10.

Evalúa el rendimiento del modelo mediante el estudio de los 3 valores que te devuelve el método que hemos visto para ello, que son: ``loss``, ``mae`` y ``mse``:

### Prediciendo

Ahora, probemos el modelo realizando una predicción de los primeros 10 valores y represéntalos en una gráfica frente a sus valores reales, es decir, uno en cada eje:

### EJERCICIO: Mejora el modelo

Aunque estemos ante muy pocas muestras, y el verdadero poder de las redes neuronales se observa con grandes volúmenes de datos, vamos a tratar de mejorar los resultados.

Modifica algunos de los parámetros utilizados en este modelo para intentar mejorar los resultados. Hay muchas posibilidades, pero empieza tocando un parámtro hasta encontrar un valor adecuado y luego modiica otros hasta obtener algo que sea suficientemente bueno.

También podrías crear nuevas variables para ver si mejora el modelo.